In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import geopandas as gpd
from shapely.geometry import Point
import datetime
import requests
import pandas as pd
import os
import sys
from osgeo import gdal

# read shape files

In [10]:

# soa_shape_map_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/doc_2024-03-26_15-21-36.gridpoint_2000.shp/.gridpoint_2000.shp.shp'
sub_basin_path = '/content/drive/MyDrive/import/Central_valley/central_valley_3857.shp'

mesh_grid_path = "/content/drive/MyDrive/import/prism_mesh/prism_4km_mesh.shp"

# soa_shape_map = gpd.read_file(soa_shape_map_path)
sub_basin_map = gpd.read_file(sub_basin_path)

mesh = gpd.read_file(mesh_grid_path)

# soa_shape_map_geo = soa_shape_map.to_crs(epsg=4326)
sub_basin_map_geo = sub_basin_map.to_crs(epsg=4326)
# mesh_geo = mesh.to_crs(epsg=4326)

# sub_basin_1 = sub_basin_map_geo[sub_basin_map_geo['Subregion_'] == 1]

mesh


,col,row,lon,lat,geometry
0,717,12,-95.125000,49.416667,"POLYGON ((-95.14583 49.43750, -95.10417 49.437..."
1,716,13,-95.166667,49.375000,"POLYGON ((-95.18750 49.39583, -95.14583 49.395..."
2,717,13,-95.125000,49.375000,"POLYGON ((-95.14583 49.39583, -95.10417 49.395..."
3,718,13,-95.083333,49.375000,"POLYGON ((-95.10417 49.39583, -95.06250 49.395..."
4,719,13,-95.041667,49.375000,"POLYGON ((-95.06250 49.39583, -95.02083 49.395..."
...,...,...,...,...,...
481626,1040,609,-81.666667,24.541667,"POLYGON ((-81.68750 24.56250, -81.64583 24.562..."
481627,1041,609,-81.625000,24.541667,"POLYGON ((-81.64583 24.56250, -81.60417 24.562..."
481628,1028,610,-82.166667,24.500000,"POLYGON ((-82.18750 24.52083, -82.14583 24.520..."
481629,1029,610,-82.125000,24.500000,"POLYGON ((-82.14583 24.52083, -82.10417 24.520..."


## sample

In [15]:
clipped_mesh = gpd.clip(mesh, sub_basin_map_geo)

# mesh_center = mesh[(mesh['col'] == 71) & (mesh['row'] == 220)]
# mesh_up = mesh[(mesh['col'] == 71) & (mesh['row'] == 219)]
# mesh_down = mesh[(mesh['col'] == 71) & (mesh['row'] == 221)]
# mesh_right = mesh[(mesh['col'] == 72) & (mesh['row'] == 220)]
# mesh_left = mesh[(mesh['col'] == 70) & (mesh['row'] == 220)]

clipped_mesh

<ipython-input-15-9fa8d20a8291>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

  clipped_mesh = gpd.clip(mesh, sub_basin_map_geo)


,col,row,lon,lat,geometry
367931,145,360,-118.958333,34.916667,"POLYGON ((-118.93750 34.93750, -118.93750 34.9..."
367933,147,360,-118.875000,34.916667,"POLYGON ((-118.88363 34.93750, -118.89583 34.9..."
367930,144,360,-119.000000,34.916667,"POLYGON ((-118.97917 34.93750, -118.97917 34.9..."
367932,146,360,-118.916667,34.916667,"POLYGON ((-118.89583 34.93750, -118.89583 34.9..."
366865,149,359,-118.791667,34.958333,"POLYGON ((-118.79617 34.97917, -118.81250 34.9..."
...,...,...,...,...,...
211328,66,223,-122.250000,40.625000,"POLYGON ((-122.24930 40.64583, -122.22917 40.6..."
211329,67,223,-122.208333,40.625000,"POLYGON ((-122.22917 40.60417, -122.22917 40.6..."
211327,65,223,-122.291667,40.625000,"POLYGON ((-122.27083 40.64583, -122.27083 40.6..."
210092,65,222,-122.291667,40.666667,"POLYGON ((-122.27083 40.64583, -122.29115 40.6..."


## create a point layer and list of coordinates

In [27]:
# copy poly to new GeoDataFrame
points = clipped_mesh.copy()
# change the geometry
points.geometry = points['geometry'].centroid
# same crs
points.crs =clipped_mesh.crs
points.head()

<ipython-input-27-165ae3f21554>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points.geometry = points['geometry'].centroid


,col,row,lon,lat,geometry
367931,145,360,-118.958333,34.916667,POINT (-118.95388 34.92906)
367933,147,360,-118.875000,34.916667,POINT (-118.89177 34.93587)
367930,144,360,-119.000000,34.916667,POINT (-118.98272 34.93581)
367932,146,360,-118.916667,34.916667,POINT (-118.92105 34.92998)
366865,149,359,-118.791667,34.958333,POINT (-118.80706 34.97626)


In [29]:


# Create a list of points with latitude and longitude
# points = [
#     Point(lon, lat) for lon, lat in [
#         (mesh_center["lon"].item(), mesh_center["lat"].item()),
#         (mesh_up["lon"].item(), mesh_up["lat"].item()),
#         (mesh_down["lon"].item(), mesh_down["lat"].item()),
#         (mesh_right["lon"].item(), mesh_right["lat"].item()),
#         (mesh_left["lon"].item(), mesh_left["lat"].item())
#         # Add more points as needed
#     ]
# ]

# Create a GeoDataFrame from the list of points
# shape_four = gpd.GeoDataFrame(geometry=points)

lat_lon_tuples_california = list(zip(points.geometry.y, points.geometry.x))
len(lat_lon_tuples_california)

3314

In [ ]:
# clipped_points = gpd.clip(soa_shape_map_geo, sub_basin_1)

# lat_lon_tuples = list(zip(clipped_points.geometry.y, clipped_points.geometry.x))


# generate time series

In [30]:


def generate_time_series(start_date, end_date, freq):
    """
    Generate a time series of dates in the format "YYYYMMDD".

    Args:
        start_date (str): The start date in the format "YYYY-MM-DD".
        end_date (str): The end date in the format "YYYY-MM-DD".
        freq (str): The frequency of the time series. Possible values are
            "D" (daily), "W" (weekly), "M" (monthly), etc.

    Returns:
        list: A list of dates in the format "YYYYMMDD".
    """
    date_series = []
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start

    while current_date <= end:
        date_series.append(current_date.strftime("%Y%m%d"))
        current_date += datetime.timedelta(days=1) if freq == "D" else datetime.timedelta(weeks=1) if freq == "W" else datetime.timedelta(days=30.4375)

    return date_series

# function of download prism data

In [33]:


def download_file(url):
  """Downloads a file from the given URL and saves it locally."""
  local_filename = f"/content/drive/MyDrive/export/prism/{url.split('/')[-1]}"
  # NOTE the stream=True parameter below
  with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
      for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)
  return local_filename


def download_prism_data(lat_lon_tuples, output_folder):
  """Downloads PIRISM data for a list of latitude and longitude tuples.

  Args:
    lat_lon_tuples: A list of tuples containing (latitude, longitude) coordinates.
    output_folder: The folder where downloaded data files will be saved.
  """
  session = requests.Session()
  base_url = "https://prism.oregonstate.edu/explorer/dataexplorer/rpc.php"
  data = {
    "spares": "4km",
    "interp": "0",
    "stats": "ppt tmin tmean tmax",  # You can customize the variables here
    "units": "si",
    "range": "daily",
    "start": "19910101",
    "end": "20201231",
    "stability": "stable",
    "call": "pp/daily_timeseries",
    "proc": "gridserv"
  }
  headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0",
    "DNT": "1",
    "Sec-GPC": "1",
  }

  for lat, lon in lat_lon_tuples:
    data["lat"] = str(lat)
    data["lon"] = str(lon)
    response = session.post(base_url, data=data, headers=headers)
    res_json = response.json()


    download_url = "https://prism.oregonstate.edu/explorer/tmp/" + res_json["result"]["csv"]
    filename = f"{output_folder}/{lat}_{lon}.csv"  # Customize filename format
    download_file(download_url)
    print(f"Downloaded data for ({lat}, {lon}) to {filename}")



## example

In [ ]:
# Example usage
output_folder = "/content/drive/MyDrive/export/prism"  # Change this if needed
download_prism_data(lat_lon_tuples_california, output_folder)

Downloaded data for (34.92906226268493, -118.9538819077391) to /content/drive/MyDrive/export/prism/34.92906226268493_-118.9538819077391.csv
Downloaded data for (34.93586509830011, -118.89176660527282) to /content/drive/MyDrive/export/prism/34.93586509830011_-118.89176660527282.csv
Downloaded data for (34.93581177674317, -118.98271712602246) to /content/drive/MyDrive/export/prism/34.93581177674317_-118.98271712602246.csv
Downloaded data for (34.929979594899265, -118.92104633728002) to /content/drive/MyDrive/export/prism/34.929979594899265_-118.92104633728002.csv
Downloaded data for (34.976262831426034, -118.80705582222832) to /content/drive/MyDrive/export/prism/34.976262831426034_-118.80705582222832.csv
Downloaded data for (35.001786012110195, -118.79302168805502) to /content/drive/MyDrive/export/prism/35.001786012110195_-118.79302168805502.csv
Downloaded data for (35.01103635213238, -119.32921437097426) to /content/drive/MyDrive/export/prism/35.01103635213238_-119.32921437097426.csv
Do

# use pyPRISMClimate library and FTP

In [ ]:

sys.path.append('/home/mahdi/data/pyPRISMClimate')
from pyPRISMClimate.quick_tools import *

## download daily data

In [ ]:
data = get_prism_dailys(
    variable ="tmean",
    min_date="1991-01-01",
    max_date="2020-12-31",
    dest_path="/home/mahdi/data/export"
)

### open bil file

In [ ]:


# Open the BIL file
dataset = gdal.Open("/home/mahdi/data/export/PRISM_tmean_stable_4kmD2_20201227_bil.bil")

### convert to array

In [ ]:
# Get the number of bands and data type
num_bands = dataset.RasterCount
data_type = dataset.GetRasterBand(1).DataType
# Read the data as a NumPy array
data = dataset.GetRasterBand(1).ReadAsArray()
# (Optional) Get geotransform information
geotransform = dataset.GetGeoTransform()
geotransform